<a href="https://colab.research.google.com/github/cbonnin88/meditrack/blob/main/The_Product_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 16.3 MB/s eta 0:00:00


In [17]:
%%writefile app.py
import streamlit as st
import polars as pl
import pandas as pd
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np


# PAGE CONFIG
st.set_page_config(page_title='MediTrack HQ', layout='wide')
st.title('🏥 MediTrack Product Analytics Dashboard')

# CACHED DATA LOADING & ML TRAINING
@st.cache_resource
def load_data_and_models():
  # Load Data
  df_users = pl.read_csv('meditrack_users_clean.csv')
  df_events = pl.read_csv('meditrack_events_clean.csv')
  df_subs = pl.read_csv('meditrack_subs_clean.csv')

  # --- PREP DATA FOR CHURN MODEL ---
  event_counts = df_events.group_by('user_id').len().rename({'len':'total_events'})
  df_ml=(
      df_users.join(df_subs,on='user_id')
      .join(event_counts, on='user_id', how='left')
      .with_columns(pl.col('total_events').fill_null(0))
      .to_pandas()
  )

  # Training the model
  features = ['age','monthly_fee','total_events']
  X = df_ml[features].fillna(0)
  y= df_ml['churned']

  model = RandomForestClassifier(n_estimators=50, random_state=42)
  model.fit(X,y)

  return df_users, df_events, df_subs, df_ml, model

# Loading everything
df_users, df_events, df_subs,df_ml, churn_model = load_data_and_models()


# --- SIDEBAR FILTER ---
st.sidebar.header('Filter Data')
selected_channel = st.sidebar.multiselect(
    'Acquisition Channel',
    options=df_users['acquisition_channel'].unique().to_list(),
    default=df_users['acquisition_channel'].unique().to_list()
)

# Filter Logic
filtered_users = df_users.filter(pl.col('acquisition_channel').is_in(selected_channel))

# --- TAB SETUP ---
tab1,tab2,tab3 = st.tabs(["📊 Business Overview", "🔮 Churn Predictor", "🩺 Doctor Recommender"])

# --- TAB 1: BUSINESS OVERVIEW ---
with tab1:
  # Row 1: KPI Cards
  col1,col2,col3 = st.columns(3)
  col1.metric('Total Users',filtered_users.height)

  active_subs = df_subs.filter(pl.col('churned')== 0).height
  col2.metric('Active Subscriptions',active_subs)

  avg_fee = df_subs['monthly_fee'].mean()
  col3.metric('Avg Monthly Fee', f'€{avg_fee:,.2f}')

  # Row 2: Charts
  st.markdown('---')
  c1,c2 = st.columns(2)

with c1:
    st.subheader('User Age Distribution')
    fig_age = px.histogram(
        filtered_users.to_pandas(),
        x='age',
        title='Age Demographics',
        labels= {'age':'Age', 'count':'Number of Users'},
        color_discrete_sequence=['#AEC6CF']
    )
    st.plotly_chart(fig_age,use_container_width=True)

with c2:
  st.subheader('Event Growth')
  daily_events = df_events.group_by('event_timestamp').len().sort('event_timestamp')
  fig_growth = px.line(
      daily_events.to_pandas(),
      x='event_timestamp',
      y='len',
      title='Daily Events',
      labels= {'event_timestamp':'Event Timestamp','len':'Length'},
      color_discrete_sequence=['#77DD77']
  )
  st.plotly_chart(fig_growth,use_container_width=True)

# --- TAB 2: CHURN PREDICTOR ---
with tab2:
  st.header('Predict User Churn')
  st.write('Adjust the sliders to see if a hypothetical user is at risk')

  # Input Form
  c1,c2,c3 = st.columns(3)
  in_age = c1.slider('User Age',18,80,30)
  in_fee = c2.selectbox('Monthly Fee', [0.0,10.95,20.95])
  in_events = c3.slider('Total Events (Engagement)',0,100,5)

  # Prediction Logic
  if st.button('Predict Risk'):
    # Create a tiny dataframe for the model
    input_data = pd.DataFrame([[in_age, in_fee, in_events]], columns=['age','monthly_fee','total_events'])
    prediction = churn_model.predict(input_data)[0]
    prob = churn_model.predict_proba(input_data)[0][1]

    if prediction == 1:
      st.error(f'⚠️ High Churn Risk! (Probability: {prob:.2%})')
      st.markdown('**Recommendation:** Send a 20% discount email immediately.')
    else:
        st.success(f"✅ Safe User. (Churn Probability: {prob:.2%})")


# --- TAB 3: RECOMMENDATIONS ---

with tab3:
  st.header('Doctor Recommendation Engine')
  st.write('Select a User ID to see presonalized doctor suggestions')

  # Simple dropdown of first 10 users for demo
  sample_users = df_users['user_id'].head(10).to_list()
  target_user = st.selectbox('Select User', sample_users)

  if st.button('Generate Recommendations'):
    st.info(f'Finding similar Users to {target_user}')

    # Fake output for the UI demo
    st.markdown(f'### Top Picks for this User:')
    cols = st.columns(3)
    cols[0].success('Dr. Dupont (Cardiology)')
    cols[1].success('Dr. OBrady (Diagnostics)')
    cols[2].success('Dr. Jones (General)')

Overwriting app.py


In [18]:
import subprocess
import time

!wget -q -O - https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb > cloudflared.deb
!dpkg -i cloudflared.deb

print("Starting Streamlit...")
subprocess.Popen(["streamlit", "run", "app.py"])

# Give it 3 seconds to start
time.sleep(3)

# Run Cloudflare Tunnel
# This searches the log file for the unique URL and prints it
!streamlit run app.py & cloudflared tunnel --url http://localhost:8501

(Reading database ... 121717 files and directories currently installed.)
Preparing to unpack cloudflared.deb ...
Unpacking cloudflared (2025.11.1) over (2025.11.1) ...
Setting up cloudflared (2025.11.1) ...
Processing triggers for man-db (2.10.2-1) ...
Starting Streamlit...
2025-12-04T13:08:57Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-04T13:08:57Z INF Requesting new quick Tunnel on trycloudflare.com...



  You can now view your Streamlit app in yo